In [1]:
! pip install trulens_eval openai langchain langchain-openai langchain_community faiss-cpu bs4 tiktoken


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()


'sk-None-kfGynoO71rXwi6WfpBZ2T3BlbkFJwMyKvfpmhW29Z9fbJlsE'

In [3]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain import hub
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

WARNI [langchain_community.utils.user_agent] USER_AGENT environment variable not set, consider setting it to identify your requests.


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [4]:
!pip3 install -qU langchain-community beautifulsoup4



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import re
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup


def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    return re.sub(r"\n\n+", "\n\n", soup.text).strip()


loader = RecursiveUrlLoader("https://en.wikipedia.org/wiki/Illuminati", extractor=bs4_extractor)
docs = loader.load()
print(docs[0].page_content)

Illuminati - Wikipedia

Jump to content

Main menu

Main menu
move to sidebar
hide

		Navigation
	

Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate

		Contribute
	

HelpLearn to editCommunity portalRecent changesUpload file

Search

Search

Appearance

Create account

Log in

Personal tools

 Create account Log in

		Pages for logged out editors learn more

ContributionsTalk

Contents
move to sidebar
hide

(Top)

1
History

Toggle History subsection

1.1
Origins

1.2
Transition

1.3
Reform

1.3.1
Adolph Knigge

1.3.2
Internal problems

1.3.3
New system

1.4
Attempts at expansion

1.4.1
Convent of Wilhelmsbad

1.4.2
Aftermath of Wilhelmsbad

1.5
Zenith

1.6
Conflict with Rosicrucians

1.7
Internal dissent

1.8
Decline

2
Barruel and Robison

3
Modern Illuminati

4
Legacy

5
References

Toggle References subsection

5.1
Works cited

6
Further reading

7
External links

Toggle the table of contents

Illuminati

74 languages

AfrikaansAlemannischالعربيةAsturian

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter and embeddings
text_splitter = RecursiveCharacterTextSplitter()
embeddings = OpenAIEmbeddings()

# Assuming docs is a list of documents
documents = text_splitter.split_documents(docs)

# Create FAISS vector store
texts = [doc.page_content for doc in documents]
embedding_results = embeddings.embed_documents(texts)

# Ensure embedding_results is not empty and has correct shape
print(len(embedding_results), len(embedding_results[0]))

vectorstore = FAISS.from_texts(texts, embeddings)


28 1536


In [7]:
pip install langchainhub


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
pip install -U langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
rag_chain.invoke("Which notable literary figures and nobility were attracted to the Illuminati?")

'Notable literary figures and nobility attracted to the Illuminati included Johann Wolfgang von Goethe, Johann Gottfried Herder, and the reigning Duke of Gotha and of Weimar. The Illuminati also had a role in Gothic literature, with readers having a "scandalous vogue for German tales of the Illuminati." Mary Shelley\'s familiarity with the early anti-Illuminati text influenced her work in Frankenstein.'

In [13]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on_input_output()
)
# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.first.steps__.context.first.invoke.rets[:].page_content .


In [14]:
tru_recorder = TruChain(rag_chain,
    app_id='ChainIllu_ChatApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [16]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("Which notable literary figures and nobility were attracted to the Illuminati?")

display(llm_response)

"Johann Wolfgang von Goethe, Johann Gottfried Herder, and the reigning Duke of Gotha and of Weimar were notable literary figures and nobility attracted to the Illuminati. The society's goals included opposing superstition, obscurantism, religious influence over public life, and abuses of state power. The Illuminati were outlawed in the late 18th century but continued to attract members from various professional classes and powerful benefactors."

In [17]:
tru.get_leaderboard()

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
ChainIllu_ChatApplication,0.0,0.5,1.0,2.5,0.004984


In [18]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, "Context Relevance")

,question,context,ret
0,Which notable literary figures and nobility we...,Books and comics[edit]\nGothic literature had ...,0.8
1,Which notable literary figures and nobility we...,Zenith\nAlthough their hopes of mass recruitme...,0.9
2,Which notable literary figures and nobility we...,"Barruel and Robison\nBetween 1797 and 1798, Au...",0.8
3,Which notable literary figures and nobility we...,18th-century Bavarian secret society\nThis art...,0.9


In [19]:
from trulens_eval.guardrails.langchain import WithFeedbackFilterDocuments

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = (
    Feedback(provider.context_relevance)
)

filtered_retriever = WithFeedbackFilterDocuments.of_retriever(
        retriever=retriever,
        feedback=f_context_relevance_score,
        threshold=0.75
    )

rag_chain = (
    {"context": filtered_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication_Filtered',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

with tru_recorder as recording:
    llm_response = rag_chain.invoke("What is Task Decomposition?")

display(llm_response)

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks. This approach helps in organizing and prioritizing the steps needed to complete the overall task efficiently. By dividing the task into smaller components, it becomes easier to assign responsibilities and track progress.'

In [ ]:
last_record = recording.records[-1]

from trulens_eval.utils.display import get_feedback_result
get_feedback_result(last_record, "Context Relevance")

""


In [ ]:
# launch on http://localhost:8501
tru.run_dashboard()


Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


ModuleNotFoundError: 
ipython,ipywidgets packages are required for using TruLens-Eval in a notebook.
You should be able to install them with pip:

    ```bash
    pip install "ipython>=8.12.0" "ipywidgets>=8.0.6"
    ```


In [ ]:
pip install "ipython>=8.12.0" "ipywidgets>=8.0.6"



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


ModuleNotFoundError: 
ipython,ipywidgets packages are required for using TruLens-Eval in a notebook.
You should be able to install them with pip:

    ```bash
    pip install "ipython>=8.12.0" "ipywidgets>=8.0.6"
    ```


In [ ]:
# The results of the feedback functions can be rertrieved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

NameError: name 'rec' is not defined

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Groundedness,Answer Relevance,Groundedness_calls,Answer Relevance_calls,Context Relevance_calls,Context Relevance,latency,total_tokens,total_cost
0,Chain1_ChatApplication_Filtered,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_629a319c4ebf6ea4fd84fd40d18d4b6b,"""What is Task Decomposition?""","""Task decomposition is the process of breaking...",-,"{""record_id"": ""record_hash_629a319c4ebf6ea4fd8...","{""n_requests"": 6, ""n_successful_requests"": 21,...","{""start_time"": ""2024-07-23T11:53:40.490987"", ""...",2024-07-23T11:53:43.153031,0.0,1.0,"[{'args': {'source': [], 'statement': 'Task de...",[{'args': {'prompt': 'What is Task Decompositi...,[],NaN,2,14935,0.022383
1,Chain1_ChatApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_d32c7c17ef1fff0b34f597cfcddaf079,"""What is Task Decomposition?""","""Task Decomposition is a technique that breaks...",-,"{""record_id"": ""record_hash_d32c7c17ef1fff0b34f...","{""n_requests"": 2, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-07-23T11:53:09.411067"", ""...",2024-07-23T11:53:11.346155,1.0,0.9,[{'args': {'source': ['Fig. 1. Overview of a L...,[{'args': {'prompt': 'What is Task Decompositi...,[{'args': {'question': 'What is Task Decomposi...,0.45,2,3306,0.004968


In [ ]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Chain1_ChatApplication,1.0,0.9,0.45,2.0,0.004968
Chain1_ChatApplication_Filtered,0.0,1.0,NaN,2.0,0.022383
